In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import os

path = os.getcwd()

## Cantilever and simulation paramters

In [2]:
R = 10.0e-9  #radius of curvature of the parabolic tip apex
startprint = 0.0
simultime = 1200.0e-6 #total simulation time
fo1 =20.0e3  #cantilever 1st mode resonance frequency

omega = 2.0*np.pi*fo1
period1 = 1.0/fo1  #fundamental period
to =7.0*period1   #centered time of the sinc excitation
fo2 = 145.9e3 #Hz, calculated from Sader
fo3 = 429.0e3 #Hz, calculated from Sader's method
k_m2 = 9.822 #N/m calculated from Sader's method
k_m3 = 76.99 #N/m calculated from Sader's method
Q1 = 2.116 #calculated from Sader's method
Q2 = 4.431 #calculated from Sader's method
Q3 = 6.769 #calculated from Sader's method
BW = 2.5*fo1*2.0  #excitation bandwith of sinc function
k_m1 =  0.25 #cantilever's 1st mode stiffness


period2 = 1.0/fo2
period3 = 1.0/fo3
dt= period3/1.0e4 #simulation timestep
printstep = period3/100.0 #timestep in the saved time array

## Sample parameters, SLS-VOigt configuration

In [3]:
nu = 0.3
Gg_v = np.array([1.0e6,10.0e6,100.0e6,1.0e9,10.0e9]) #Glassy modulus in the Voigt-SLS configuration reported in the grid
tau_v = np.array([0.01/omega, 0.1/omega, 1.0/omega, 10.0/omega, 100.0/omega]) #retardation time reported in simulation grid
G_v = 1.0e-1/(1.2*R) #modulus of the spring in the Voigt unit that is in series with the upper spring
Jg = 1.0/Gg_v #glassy compliance
J = 1.0/G_v #compliance of the spring in the Voigt unit that is in series with the upper spring
Je = J+Jg  #equilibrium compliance of the SLS-VOigt model

## Continuum simulations MDR with thermal noise

In [6]:
from AFM_sinc import MDR_SLS_sinc_noise_sader, brownian_noise_sader

Temp = 273.16 + 25
dt = period3/1.0e4
printstep = period3/1.0e2
Fb1, Fb2, Fb3, _ = brownian_noise_sader(Temp, fo1, k_m1, Q1, dt, simultime)

In [7]:
os.chdir(path + '/NoiseA6')
print os.getcwd()


A = -6.8e-9 #amplitude of the sinc excitation
zb = 15.0e-9  #cantilever equilibrium position

np.savetxt('Simul_parameters.txt', np.array([zb*1.0e9, A*1.0e9, fo1*1.0e-3, k_m1, Q1]), header = 'Zb(nm)\tAmp(nm)\tfo1(kHz)\tQ1', delimiter = '\t')

C:\Users\enrique\GithubRepositories\SingleImpact_Simulations\NoiseA6


In [8]:
mdr_jit = jit()(MDR_SLS_sinc_noise_sader)

A = -6.8e-9 #amplitude of the sinc excitation
zb = 15.0e-9  #cantilever equilibrium position

dmax = 5.0e-9
Ndy = 1000


for i in range(len(Gg_v)):
    for j in range(len(tau_v)):
        #Transfering to SLS-Maxwell configuration for the MDR algorithm
        Ge = 1.0/(Je[i])
        G = J/(Jg[i]*Je[i]) 
        Gg = (G+Ge)
        tau_m = tau_v[j]*(Ge/Gg)          
        G_mpa = G/1.0e6
        Ge_mpa = Ge/1.0e6
        tau_us = tau_m*1.0e6
        print('G: %.3f MPa, Ge: %.3f MPa, tau: %.5f us'%(G_mpa, Ge_mpa, tau_us))
        %time t, tip, Fts, ca, sample, Fsinc, z1, z2, z3 = mdr_jit(A, to , BW, G, tau_m, R, dt, startprint, simultime, fo1, k_m1, zb, Fb1, Fb2, Fb3, printstep, Ge, Q1, Q2, Q3, nu, Ndy, dmax, Temp)
        np.savetxt('wNoise_Gg%d_tau%d.txt'%(i+1,j+1), np.array((t-to, Fts, Fsinc, z1, z2, z3)).T, header = 'time(s)\tFts(N)\tSinc_Force(N)\tz1(m)\tz2(m)\tz3(m)', delimiter = '\t')
        print('wNoise_Gg%d_tau%d.txt'%(i+1,j+1))

G: 0.107 MPa, Ge: 0.893 MPa, tau: 0.07105 us
Wall time: 10min 45s
wNoise_Gg1_tau1.txt
G: 0.107 MPa, Ge: 0.893 MPa, tau: 0.71051 us
Wall time: 4min 48s
wNoise_Gg1_tau2.txt
G: 0.107 MPa, Ge: 0.893 MPa, tau: 7.10513 us
Wall time: 48.9 s
wNoise_Gg1_tau3.txt
G: 0.107 MPa, Ge: 0.893 MPa, tau: 71.05131 us
Wall time: 49 s
wNoise_Gg1_tau4.txt
G: 0.107 MPa, Ge: 0.893 MPa, tau: 710.51314 us
Wall time: 49.1 s
wNoise_Gg1_tau5.txt
G: 5.455 MPa, Ge: 4.545 MPa, tau: 0.03617 us
Wall time: 10min 43s
wNoise_Gg2_tau1.txt
G: 5.455 MPa, Ge: 4.545 MPa, tau: 0.36172 us
Wall time: 4min 48s
wNoise_Gg2_tau2.txt
G: 5.455 MPa, Ge: 4.545 MPa, tau: 3.61716 us
Wall time: 49 s
wNoise_Gg2_tau3.txt
G: 5.455 MPa, Ge: 4.545 MPa, tau: 36.17158 us
Wall time: 49 s
wNoise_Gg2_tau4.txt
G: 5.455 MPa, Ge: 4.545 MPa, tau: 361.71578 us
Wall time: 49.1 s
wNoise_Gg2_tau5.txt
G: 92.308 MPa, Ge: 7.692 MPa, tau: 0.00612 us
Wall time: 10min 43s
wNoise_Gg3_tau1.txt
G: 92.308 MPa, Ge: 7.692 MPa, tau: 0.06121 us
Wall time: 4min 47s
wNoise_